# Copper Energy Demands
This journal documents the energy demands of mining, refining, drawing and recycling of copper for use inside the PV cell. Probably also applies to copper in wiring (not yet considered)

In [8]:
import numpy as np
import pandas as pd
import os,sys
from pathlib import Path
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = (8, 4)
cwd = os.getcwd() #grabs current working directory

supportMatfolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'baselines' / 'SupportingMaterial')
baselinesFolder = str(Path().resolve().parent.parent.parent / 'PV_ICE' / 'baselines')

### Mining Energy

In [16]:
os.path.join(supportMatfolder+"\energy-input-copper-mining.csv")

'C:\\Users\\hmirletz\\Documents\\GitHub\\PV_ICE\\PV_ICE\\baselines\\SupportingMaterial\\energy-input-copper-mining.csv'

In [17]:
cu_mining_raw = pd.read_csv(os.path.join(supportMatfolder+"\energy-input-copper-mining.csv"), index_col='year')
cu_mining_raw.dropna(how='all')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\hmirletz\\Documents\\GitHub\\PV_ICE\\PV_ICE\\baselines\\SupportingMaterial\\energy-input-copper-mining.csv'

In [ ]:
plt.scatter(cu_mining_raw.index, cu_mining_raw['E_CuMining_kWhpkg'], marker='o')
plt.title('CED_Cu_kWhpkg')
plt.ylabel('kWh/kg')

In [ ]:
#drop ones that include more than just mining and concentration.
cu_mining_raw.loc[1975] = np.nan
cu_mining_raw.loc[2010] = np.nan
cu_mining_raw.loc[2011] = np.nan


In [ ]:
plt.scatter(cu_mining_raw.index, cu_mining_raw['E_CuMining_kWhpkg'], marker='o')
plt.xlim(1995,2020)
plt.title('CED_Cu_kWhpkg')
plt.ylabel('kWh/kg')

This is a scatter plot. We're going to use Farjana et al 2019 because of the detail provided in the datapoint. 

### CED
CED would include mining, so we might subtract the mining energy from the final CED trend.

In [ ]:
cu_CED_raw = pd.read_csv(cwd+"/../../../PV_ICE/baselines/SupportingMaterial/energy-input-copper-CED.csv",
                                     index_col='year')
cu_CED_raw.dropna(how='all')

In [ ]:
plt.scatter(cu_CED_raw.index, cu_CED_raw['CED_Cu_kWhpkg'], marker='o')
plt.title('CED_Cu_kWhpkg')
plt.ylabel('kWh/kg')

In [ ]:
#drop the low outlier
cu_CED_raw.loc[1999] = np.nan
#drop the 2009 Nuss datapoint because it includes recycled content
cu_CED_raw.loc[2009] = np.nan
#drop 2012 EU survey, includes scrap content
cu_CED_raw.loc[2012] = np.nan

The remaining lower energy points are associated with the difference between Hydro and Pyro metallurgy. 

In [ ]:
cu_CED_raw.loc[2000:2001,['Notes']]

In [ ]:
cu_CED_raw.loc[2006:2007,['Notes']]

In [ ]:
#find pyro data
cu_CED_trim = cu_CED_raw.dropna(how='all')
cu_CED_trim_pyro = cu_CED_trim[cu_CED_trim['Notes'].str.contains('pyro')]
cu_CED_trim_pyro#.index

In [ ]:
#find hydro data
cu_CED_trim_hydro = cu_CED_trim[cu_CED_trim['Notes'].str.contains('hydro')]
cu_CED_trim_hydro#.index

In [ ]:
plt.scatter(cu_CED_raw.index, cu_CED_raw['CED_Cu_kWhpkg'], marker='o', label='alldata', color='black', s=40)
plt.scatter(cu_CED_trim_pyro.index, cu_CED_trim_pyro['CED_Cu_kWhpkg'], marker='^', label='pyro', color='orange', s=30)
plt.scatter(cu_CED_trim_hydro.index, cu_CED_trim_hydro['CED_Cu_kWhpkg'], marker='o', label='hydro', color='skyblue', s=20)

plt.ylim(0,)
plt.title('CED_Cu_kWhpkg')
plt.ylabel('kWh/kg')
plt.legend()

Our method will be to take the average of the hydro and average of the pyro CEDs, then use marketshare of each technology over time for 1995 through 2050. Then we will use the projected energy from Rotzer for 2050 and interpolate. This neglects the detail of increasing energy demands for decreasing ore grade - however, the correlation from Northey is <0.4 for the full process and average global ore grade over time has been hard to find and is not fully representative of energy demands.

In [ ]:
#hydro
CED_hydro = cu_CED_trim_hydro['CED_Cu_kWhpkg'].mean()

In [ ]:
#pyro
cu_CED_trim_pyro.loc[2050]=np.nan #drop the 2050 value (will use to interpolate later)
CED_pyro = cu_CED_trim_pyro['CED_Cu_kWhpkg'].mean()
CED_pyro

In [ ]:
#import marketshare
mrktshr_pyrohydro_raw = pd.read_excel(cwd+"/../../../PV_ICE/baselines/SupportingMaterial/energy-inputs-copper-maths.xlsx",
              sheet_name='pyrovshydro', index_col=0, header=[0])

In [ ]:
mrktshr_pyrohydro = mrktshr_pyrohydro_raw[['% PYRO','% HYDRO']]

plt.stackplot(mrktshr_pyrohydro.index, mrktshr_pyrohydro['% PYRO'], mrktshr_pyrohydro['% HYDRO'], colors=['orange','skyblue'])
plt.legend(mrktshr_pyrohydro.columns, loc='lower right')

In [ ]:
pyro_mrktshr_CED = mrktshr_pyrohydro['% PYRO']*CED_pyro
pyro_mrktshr_CED[0:5]

In [ ]:
hydro_mrktshr_CED = mrktshr_pyrohydro['% HYDRO']*CED_hydro
hydro_mrktshr_CED[0:5]

In [ ]:
11.727135+1.439028

In [ ]:
CED_mrktshr = pd.DataFrame(pyro_mrktshr_CED+hydro_mrktshr_CED)
plt.plot(CED_mrktshr)
plt.ylim(10,20)

Now in 2050 we have two projections of energy demand in 2050. There is another projection from Elshkaki 2016, but this is older and nearly double the other two estimates. We will average the two 2050 projection estimates and interpolate to that value.

In [ ]:
CED_Cu_tofill = pd.concat([CED_mrktshr.loc[1995:],cu_CED_raw.loc[2022:,'CED_Cu_kWhpkg']])
#create the new 2050 value
CED_2050_mean = cu_CED_raw.loc[2022:,'CED_Cu_kWhpkg'].dropna(how='all').mean()
#CED_2050_mean
#set the 2049 to nan and 2050 to the new mean
CED_Cu_tofill.loc[2049]=np.nan
CED_Cu_tofill.loc[2050]=CED_2050_mean
#interpolate
CED_Cu_filled = CED_Cu_tofill.interpolate()

In [ ]:
plt.plot(CED_Cu_filled)

### Add the Energy to turn Cu cathode in to Cu wire